## Fairness LLM

For detailed documentation, including API references, tutorials, and best practices, please visit our comprehensive documentation site:

[TrustEval Documentation](https://trustgen.github.io/trustgen_docs/)

In [1]:
import os
import sys
import nest_asyncio
nest_asyncio.apply()

## conmment this line if you don't need to use proxy
os.environ['http_proxy'] = 'http://127.0.0.1:7890' 
os.environ['https_proxy'] = 'http://127.0.0.1:7890'  

parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)

base_dir = os.path.abspath("./test/fairness_llm/")

### Metadata Curator


In [ ]:
from trusteval.src.download import download_metadata
download_metadata('fairness_llm', base_dir)

In [ ]:
from trusteval.dimension.fairness.fairness_llm import pipeline
pipeline.run(base_dir=base_dir,subset=['preference', 'disparagement'])

### Contexual Variator

In [ ]:
from trusteval import contextual_variator_cli
import shutil

source_config = os.path.join(parent_dir,"trusteval","dimension","fairness",'fairness_llm','file_config_fairness.json')
target_config = os.path.join(base_dir,"file_config.json")
if os.path.exists(source_config):
    shutil.copy2(source_config, target_config)
    print(f"Successfully copied file_config to {target_config}")
else:
    print("Warning: Source file_config not found")

contextual_variator_cli(
    dataset_folder=base_dir
)

### Response Generator

In [ ]:
from trusteval import generate_responses

request_type = ['llm']
async_list = ['gpt-4o']
await generate_responses(
    data_folder=base_dir,
    request_type=request_type,
    async_list=async_list
)

### Judge Processor

In [ ]:
from trusteval.src.evaluation import judge_responses
async_list = ['gpt-4o-mini']
target_models = async_list
judge_type = 'llm'
judge_model = ['gpt-4o-mini']
config_path = os.path.join(parent_dir, 'trusteval/src/config/judge_prompt.yaml')

await judge_responses(
    data_folder=base_dir,
    async_judge_model=judge_model,
    target_models=target_models,
    judge_type=judge_type,
)

### Metrics

In [ ]:
from trusteval.src.evaluation import lm_metric

lm_metric(
    base_dir=base_dir,
    aspect="fairness_llm",
    model_list=async_list
)

### Report

In [ ]:
import nest_asyncio
nest_asyncio.apply()
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)

from trusteval import report_generator

report_generator(
    base_dir=base_dir,
    aspect='Fairness',
    model_type='llm',
)